In [2]:
%pip install confluent-kafka
%pip install pyspark
%pip install pymongo
%pip install requests
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, month, year
from pyspark.sql.functions import split, col, regexp_replace
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import confluent_kafka as ck
import random
import time
import requests
import csv
from io import StringIO
import pandas as pd
from datetime import datetime, timedelta, date
import calendar
import re
import time

# Kafka

## Config Kafka

In [3]:
config = {}
# Required connection configs for Kafka producer, consumer, and admin
config["bootstrap.servers"]="pkc-60py3.europe-west9.gcp.confluent.cloud:9092"
config["security.protocol"]="SASL_SSL"
config["sasl.mechanisms"]="PLAIN"
config["sasl.username"]="GDHUJVNTILOC36TK"
config["sasl.password"]="sQ4OhM7DZbJrDx4j00B6IUuxRG8cQmjM1fSZg/foCiKD1RHCCU1DYIGCj+ouY02O"

# Best practice for higher availability in librdkafka clients prior to 1.7
config["session.timeout.ms"]=45000

config["group.id"] = "python-group-1"
config["auto.offset.reset"] = "earliest"

"""
# Required connection configs for Confluent Cloud Schema Registry
schema.registry.url="https://psrc-9jzo5.us-central1.gcp.confluent.cloud"
basic.auth.credentials.source="USER_INFO"
basic.auth.user.info="V62ZHMWOWECRACCJ:no7BCD6mPA85DsECXfSKijlOJpHDAYcemUiIZAjk7d49AUuEwdJKXCquZZm4atTN"
"""


'\n# Required connection configs for Confluent Cloud Schema Registry\nschema.registry.url="https://psrc-9jzo5.us-central1.gcp.confluent.cloud"\nbasic.auth.credentials.source="USER_INFO"\nbasic.auth.user.info="V62ZHMWOWECRACCJ:no7BCD6mPA85DsECXfSKijlOJpHDAYcemUiIZAjk7d49AUuEwdJKXCquZZm4atTN"\n'

# Consumer

In [4]:
from io import StringIO
consumer = ck.Consumer(config)
consumer.subscribe(["meteo_data"])
try:
     while True:
          msg = consumer.poll(1.0)
          if msg is not None and msg.error() is None:
               df = pd.DataFrame(columns=["station_id", "dh_utc", "temperature", "vent_moyen", "pluie_1h"])
               input_strings = msg.value().decode('utf-8').split('\n')
               for input_string in input_strings:
                    new_data = pd.read_csv(StringIO(input_string), sep=';', names=df.columns)
                    df = pd.concat([df, new_data], ignore_index=True)
               display(df)
except KeyboardInterrupt:
     pass
finally:
     consumer.close()


C:\Users\Julian\AppData\Local\Temp\ipykernel_5108\2184261065.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_data], ignore_index=True)


# Creation Session Spark

In [5]:
spark = SparkSession.builder.master("local[*]").appName("YourAppName").config("spark.driver.memory", "4g").config("spark.memory.offHeap.enabled","true").config("spark.memory.offHeap.size","10g").config("spark.executor.cores", "4").getOrCreate() 

# Calcul des valeurs moyennes sur le mois

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg

def calcul_vals_moyennes(df):

    # Convert pandas DataFrame to Spark DataFrame
    data_df = spark.createDataFrame(df, schema=["station_id", "dh_utc", "temperature", "vent_moyen", "pluie_1h"])

    # Extraction du mois et de l'année à partir de la colonne dh_utc
    data_df = data_df.withColumn("month", month("dh_utc"))
    data_df = data_df.withColumn("month", year("dh_utc"))

    stations_csv_path = "./stations_formatted.csv"

    # Define the schema explicitly
    schema = StructType([
        StructField("id_station", IntegerType(), True),
        StructField("nom_station", StringType(), True)
    ])

    # Charger le fichier de correspondance dans un DataFrame Spark
    stations_df = spark.read.option("delimiter", ":").csv(stations_csv_path, header=True, schema=schema)
    # Supprimer la ligne contenant "metadata null"
    # stations_df = stations_df.filter(~col("id_station").contains("metadata") & ~col("nom_station").contains("null"))

    # Jointure entre df_filtered et stations_df
    df_joined = data_df.join(stations_df, data_df.station_id == stations_df.id_station)


    # Grouper les données par station_id, nom_station et mois, puis calculer les moyennes
    average_df = df_joined.groupBy("station_id").agg(
        avg("pluie_1h").alias("avg_pluie_1h"),
        avg("vent_moyen").alias("avg_vent_moyen"),
        avg("temperature").alias("avg_temperature")
    )

    # On retourne les résultats
    return(average_df)
    

test = calcul_vals_moyennes(df)
test.show()

Py4JJavaError: An error occurred while calling o96.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 1.0 failed 1 times, most recent failure: Lost task 3.0 in stage 1.0 (TID 4) (DESKTOP-IQL3AGM executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoin

# Calcul moyennes sur 1 mois

In [17]:


# Group by 'station_id' and calculate overall averages
averages_df = df.groupby('station_id').agg({'temperature': 'mean', 'vent_moyen': 'mean', 'pluie_1h': 'mean'}).reset_index()

# Display the resulting DataFrame with overall averages
display(averages_df)

# Extract year and month


# Print the results
print("Year:", year)
print("Month:", month)

# IL MANQUE DES STATIONS EST CE QUE CEST A CAUSE DES NAN DANS PRODUCER

,station_id,temperature,vent_moyen,pluie_1h
0,23,4.643533,1.755216,0.625591
1,42,7.342816,4.994253,0.187931
2,000BZ,8.777260,3.657062,0.231921
3,000C5,7.828969,1.733148,0.182730
4,000CJ,2.894101,3.946489,0.225843
5,000DF,10.066239,10.323504,0.408832
6,000DR,6.914993,4.445686,0.194625
7,000EB,7.086950,7.282270,0.189220
8,000EU,6.970255,6.358782,0.181586
9,000FA,6.191513,5.912447,0.077511


Year: 2019
Month: 11
